<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# Reconnaissance de caractères manuscrits ([MNIST](http://yann.lecun.com/exdb/mnist/)) avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>

## 1 Introduction

### 1.1 Objetifs

L'[atelier](http://wikistat.fr/pdf/st-atelier-MINST.pdf) de [wikistat](wikistat.fr) propose de comparer des versions R, python et H2O. Ce calepin décline la solution en **R**.

Le site de Yann Le Cun: [MNIST DataBase](http://yann.lecun.com/exdb/mnist/), est la source des données étudiées, il  décrit précisément le problème et les modes d'acquisition. Il tient également à jour la liste des publications proposant des solutions avec la qualité de prévision obtenue. 

De façon très schématique, plusieurs stratégies sont développées dans une vaste littérature sur ces données.  
* Utiliser une méthode classique (k-nn, random forest...) sans trop raffiner mais avec des temps d'apprentissage rapide conduit à un taux d'erreur un peu supérieur à 3%.
* Ajouter  ou intégrer un pré-traitement des données permettant de recaler les images par des distorsions plus ou moins complexes.
* Construire une mesure de distance adaptée au problème, par exemple invariante par rotation, translation, puis l'intégrer dans une technique d'apprentissage classique. 
* Utiliser une méthode plus flexibles (réseau de neurones "profond", scattering) avec une optimisation fine des paramètres. 
* ...

L'**objectif** de ce calepin n'est pas de minimiser le taux d'erreur avec des méthodes sophistiquées mais d'utiliser ces données relativement volumineuses pour comparer diverses implémentations des méthodes d'apprentissage classiques. 

### 1.2 Lecture des données d'apprentissage et de test

In [1]:
# Après avoir téléchargé les données
# Fichier train.csv
# Lecture des données 
Dtrain=read.csv("mnist_train.csv",header=F)
dim(Dtrain)

[1] 60000   785

In [2]:
Ltrain=as.factor(Dtrain[,785])
Dtrain=Dtrain[,-785]

In [3]:
# Même chose avex les données de test

In [4]:
Dtest=read.csv("mnist_test.csv",header=F)
Ltest=as.factor(Dtest[,785])
Dtest=Dtest[,-785]
dim(Dtest)

[1] 10000   784

### 1.3 Exploration 

Les données ont déjà été normalisées centrées et ne présentent et sont complètes. Elles ne nécessitent pas d'autre "nettoyage" au moins rudimentaire.

Le [tutoriel](http://wikistat.fr/pdf/st-tutor3-python-scikit.pdf) d'introduction à Scikit-learn montre comment représenter les images des caractères ainsi qu'une ACP qui n'est pas reprise ici. 

On s'intéresse aux  performances de l'implémentation de k-means en R sur un tel volume.

In [12]:
t1=Sys.time()
# Problème de convergence selon l'algorithme utilisé
clas.dig=kmeans(Dtrain,10, algorithm="Forgy",iter.max=50)
t2=Sys.time()
# temps d'exécution
difftime(t2,t1)

Warning message:
: did not converge in 50 iterations

Time difference of 1.073828 mins

In [13]:
classe=clas.dig$cluster
# Homogénéité des classes
table(Ltrain, classe)

      classe
Ltrain    1    2    3    4    5    6    7    8    9   10
     0   28    1 3842  222 1597   18  152   12   40   11
     1   73    5    0   10    3 6594   11    9   12   25
     2  227   93   32  299  143  701  177   26  150 4110
     3  658   39   21 3859  786  421   38   25  153  131
     4   15    7    9    0   24  194   98 2854 2612   29
     5  508   23   48 1868 1432  714   82  434  308    4
     6   16    0   74   27  533  451 4637   11   89   80
     7   89 4234   11    5   10  472    4  271 1132   37
     8 3525   20   30 1121  148  557   40  197  173   40
     9   68  199   34  100   12  234    6 2891 2391   14

## 2 Apprentissage et prévision du test

Tester ensuite différents modèles de discrimination. Bien entendu, il s'agirait d'optimiser les paramètres des modèles mais en prévoyant de restreindre la taille de l'échantillon d'apprentissage lorque les temps de calcul sont importants...

### 2.1 $K$ plus proches voisins

Les exécutions sont proposés sur un sous échantillon de l'échantillon d'apprentissage initial. Estimer le temps d'exécution sur tout l'échantillon ou l'obtenir ... de nuit!

La complexité de l'algorithme $k$-nn est en $O(nkd)$ où $d$ est la dimension, $k$, le nombre de voisins et $n$ la taille de l'échantillon d'apprentissage.

In [5]:
# Sous échantillonnage
set.seed(11)
SousEch=sample(1:nrow(Dtrain),nrow(Dtrain)/5)
EchDtrain=Dtrain[SousEch,]
EchLtrain=Ltrain[SousEch]
dim(EchDtrain)

[1] 12000   784

In [8]:
library(class)
t1=Sys.time() 
prev.knn=knn(EchDtrain,Dtest,EchLtrain,k=10)
t2=Sys.time()
difftime(t2,t1)

Time difference of 39.75309 mins

In [11]:
# matrice de confusion
conf=table(prev.knn,Ltest)
conf

        Ltest
prev.knn    0    1    2    3    4    5    6    7    8    9
       0  970    0   21    0    1    3    8    0   12    9
       1    1 1129   27    4   16    6    5   39    8    8
       2    1    2  936    3    0    0    0    2    4    2
       3    0    1   10  964    0   22    0    1   26    6
       4    0    0    2    1  924    3    3    2    9   11
       5    2    0    0   15    0  833    3    0   21    1
       6    5    3    4    0    7   13  939    0    2    1
       7    1    0   23   11    1    3    0  964    9   20
       8    0    0    9    6    0    2    0    0  874    1
       9    0    0    0    6   33    7    0   20    9  950

In [12]:
#taux d'erreur
(sum(conf)-sum(diag(conf)))/sum(conf)

[1] 0.0517

### 2.2 Random Forest

L'algorithme random forest de la librairie `randomForest` est implémenté en fortran puis interfacé avec R. Il se montre plus efficace que celle de $k$-nn mais nettement plus long que l'implémentation de la librairie `ranger` par [Wright et Ziegler (2015)](http://arxiv.org/pdf/1508.04409v1.pdf).

#### `randomForest`

In [19]:
## Random Forest avec sous-échantillon
library(randomForest)
t1=Sys.time() 
rf.fit=randomForest(x=EchDtrain,y=EchLtrain,xtest=Dtest,ytest=Ltest,ntree=100)
# mtry par défaut (sqrt(p))
t2=Sys.time()
difftime(t2,t1)

Time difference of 57.99853 mins

In [39]:
rf.fit$test$err.rate[100,1]

Test 
0.0287

#### `ranger`

Même calcul avec l'échantillon complet mais en utilisant l'implémentation de "ranger". Seul souci, la parallélisation (*multithreading*) n'est pas possible sous Windows alors qu'elle est automatique avec un autre système. Le temps d'apprentissage pourrait être encore sensiblement amélioré.

In [6]:
library(ranger)
t1=Sys.time() 
DataF=data.frame("Ltrain"=Ltrain,Dtrain)
rf.fit=ranger(Ltrain~.,data=DataF,num.trees=100,write.forest=TRUE)
# mtry par défaut (sqrt(p)) 
t2=Sys.time()
difftime(t2,t1)

Warning message:
: package 'ranger' was built under R version 3.2.5

Growing trees.. Progress: 34%. Estimated remaining time: 1 minute, 0 seconds.
Growing trees.. Progress: 66%. Estimated remaining time: 31 seconds.


Time difference of 1.642127 mins

In [7]:
predY=predict(rf.fit,dat=Dtest)
predY$class
conf=table(predY$predictions,Ltest)
erreur=(sum(as.vector(conf))-sum(diag(conf)))/nrow(Dtest)
print(erreur)

NULL

[1] 0.031


## 3 Effet de la taille de l'échantillon d'apprentissage avec `ranger`

La procédure d'estimation du modèle puis de prévision de l'échantillon test est itérée en fonction de la taille croissante de l'échantillon d'apprentissage.

In [9]:
ntree=250
errMat=matrix(rep(0,36),nrow=12,ncol=3 )
dimnames(errMat)[[2]]=c("Taille","Temps","Erreur")
for (i in 1:12) {
  SousEch=sample(1:60000,5000*i)
  EchDtrain=Dtrain[SousEch,]
  EchLtrain=Ltrain[SousEch]
  n=dim(EchDtrain)
  t1=Sys.time() 
  DataF=data.frame("Ltrain"=EchLtrain,EchDtrain)
  rf.fit=ranger(Ltrain~.,data=DataF,num.trees=ntree,write.forest=TRUE)
  t2=Sys.time()
  errMat[i,1]=5000*i
  errMat[i,2]=difftime(t2,t1)
  predY=predict(rf.fit,dat=Dtest)
  conf=table(predY$predictions,Ltest)
  errMat[i,3]=(sum(as.vector(conf))-sum(diag(conf)))/nrow(Dtest)
}
print(errMat)

Growing trees.. Progress: 67%. Estimated remaining time: 15 seconds.
Growing trees.. Progress: 48%. Estimated remaining time: 33 seconds.
Growing trees.. Progress: 97%. Estimated remaining time: 2 seconds.
Growing trees.. Progress: 37%. Estimated remaining time: 53 seconds.
Growing trees.. Progress: 72%. Estimated remaining time: 24 seconds.
Growing trees.. Progress: 26%. Estimated remaining time: 1 minute, 26 seconds.
Growing trees.. Progress: 56%. Estimated remaining time: 47 seconds.
Growing trees.. Progress: 86%. Estimated remaining time: 14 seconds.
Growing trees.. Progress: 26%. Estimated remaining time: 1 minute, 30 seconds.
Growing trees.. Progress: 50%. Estimated remaining time: 1 minute, 3 seconds.
Growing trees.. Progress: 75%. Estimated remaining time: 31 seconds.
Growing trees.. Progress: 22%. Estimated remaining time: 1 minute, 49 seconds.
Growing trees.. Progress: 44%. Estimated remaining time: 1 minute, 17 seconds.
Growing trees.. Progress: 64%. Estimated remaining time